In [1]:
pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to c:\users\hammad\appdata\local\temp\pip-req-build-rkiiuedh
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369594 sha256=71851aae259270ba87ed0859f06c7f59a823ec6008c9d0013b13dca4bbdcf1b2
  Stored in directory: C:\Users\Hammad\AppData\Local\Temp\pip-ephem-wheel-cache-o0fjyafx\wheels\cb\a8\74\5f32d6cf0407457f0f62737b6da5c14eb86b9cac476fdf630d
Successfully built clip
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hammad\AppData\Local\Temp\pip-req-build-rkiiuedh'


In [2]:
import torch
import clip
from PIL import Image
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import json
from concurrent.futures import ThreadPoolExecutor


In [3]:
# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


In [4]:
# COCO dataset paths
COCO_IMAGE_FOLDER = r"D:\VS_Jnotepad_code\Search Enginer Project\coco_dataset\train2017_subset"
COCO_ANNOTATIONS_FILE = r"D:\VS_Jnotepad_code\Search Enginer Project\coco_dataset\annotations\annotations\captions_train2017.json"

In [5]:
# Load COCO captions
def load_coco_captions(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)
    image_id_to_captions = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        caption = ann['caption']
        if image_id not in image_id_to_captions:
            image_id_to_captions[image_id] = []
        image_id_to_captions[image_id].append(caption)
    return image_id_to_captions, data['images']

# Load and preprocess COCO images in parallel
def process_single_image(img_info, folder, captions):
    filename = img_info['file_name']
    img_path = os.path.join(folder, filename)
    image_id = img_info['id']
    
    try:
        if os.path.exists(img_path):
            image = Image.open(img_path)
            image = image.convert("RGB")  # Convert to RGB to avoid mode issues
            image = preprocess(image).unsqueeze(0).to(device)
            with torch.no_grad():
                img_embedding = model.encode_image(image)
                img_embedding /= img_embedding.norm(dim=-1, keepdim=True)
            
            text_embeds = []
            if image_id in captions:
                for caption in captions[image_id]:
                    caption_tokenized = clip.tokenize([caption]).to(device)
                    with torch.no_grad():
                        text_embedding = model.encode_text(caption_tokenized)
                        text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
                    text_embeds.append(text_embedding.cpu().numpy())
            
            text_embedding_mean = np.mean(text_embeds, axis=0) if text_embeds else None
            return filename, img_embedding.cpu().numpy(), text_embedding_mean
    except (OSError, Image.UnidentifiedImageError) as e:
        print(f"❌ Skipping corrupted image: {filename} - {str(e)}")
    return None

# Parallel processing
def load_and_preprocess_coco_images(folder, image_data, captions, num_workers=20):
    image_embeddings = {}
    text_embeddings = {}
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(lambda img_info: process_single_image(img_info, folder, captions), image_data)
    
    for result in results:
        if result is not None:
            filename, img_embedding, text_embedding = result
            image_embeddings[filename] = img_embedding
            if text_embedding is not None:
                text_embeddings[filename] = text_embedding
    
    return image_embeddings, text_embeddings

In [6]:

# Load COCO dataset
coco_captions, coco_images = load_coco_captions(COCO_ANNOTATIONS_FILE)
image_embeddings, text_embeddings = load_and_preprocess_coco_images(COCO_IMAGE_FOLDER, coco_images, coco_captions, num_workers=12)

# Save embeddings
pickle.dump(image_embeddings, open("coco_image_embeddings.pkl", "wb"))
pickle.dump(text_embeddings, open("coco_text_embeddings.pkl", "wb"))

print("✅ COCO Image and Text Embeddings Saved!")

❌ Skipping corrupted image: 000000514846.jpg - image file is truncated (19 bytes not processed)
✅ COCO Image and Text Embeddings Saved!
